In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from datetime import date, datetime

In [ ]:
INPUT_PATH = 'YOUR INPUT PATH'
df = pd.read_csv(INPUT_PATH + 'data_real_transformed.csv')
ethnicity = pd.read_csv(INPUT_PATH + 'demographics_icustayid.csv')
real = pd.read_csv(INPUT_PATH + 'data_real.csv')

In [ ]:
ethnicity = ethnicity.dropna(subset=['icustay_id'], axis=0).reset_index(drop=True)

In [ ]:
categories = []
for i in ethnicity['ethnicity']:
  if not i in categories:
    categories.append(i)

categories.sort()

In [ ]:
white = []
black = []
hispanic = []
asian = []
other = []

for category in categories:
  if category.find('WHITE') != -1:
    white.append(category)
  elif category.find('BLACK') != -1:
    black.append(category)
  elif category.find('HISPANIC') != -1:
    hispanic.append(category)
  elif category.find('ASIAN') != -1:
    asian.append(category)
  else:
    other.append(category)


In [ ]:
ethnicity = ethnicity.replace(white,'WHITE')
ethnicity = ethnicity.replace(black,'BLACK')
ethnicity = ethnicity.replace(hispanic,'HISPANIC')
ethnicity = ethnicity.replace(asian,'ASIAN')
ethnicity = ethnicity.replace(other,'OTHER')

In [ ]:
ethnicity = ethnicity.drop(ethnicity[(ethnicity['ethnicity'] != 'WHITE') & (ethnicity['ethnicity'] != 'BLACK')].index, inplace=False)

# 0 white, 1 black 
ethnicity = ethnicity.replace('WHITE', 0)
ethnicity = ethnicity.replace('BLACK', 1)

In [ ]:
ethnicity_ids = np.array(ethnicity['icustay_id'])
df = df.loc[df['icustay_id'].isin(ethnicity_ids)]
hypotension_ids = np.array(df['icustay_id'].unique())
ethnicity = ethnicity.loc[ethnicity['icustay_id'].isin(hypotension_ids)]

In [ ]:
assert any(ethnicity['icustay_id'] == df['icustay_id'].unique()), "ids do not match"

In [ ]:
ethnicity = pd.concat([ethnicity]*48).sort_index().reset_index(drop=True)
df.reset_index(drop=True, inplace=True)
df['ethnicity'] = ethnicity['ethnicity']
df.sort_values(['icustay_id', 'hour'], inplace=True)

In [ ]:
real = real.loc[real['icustay_id'].isin(hypotension_ids)]
df.reset_index(drop=True, inplace=True)
real.sort_values(['icustay_id', 'hours'], inplace=True)
real['ethnicity'] = df['ethnicity']

In [ ]:
OUTPUT_PATH = 'YOUR OUTPUT PATH'

df.to_pickle(OUTPUT_PATH + "data_real_transformed.pkl")
df.to_csv(OUTPUT_PATH + "data_real_transformed.csv", index=False)
real.to_pickle(OUTPUT_PATH + "data_real.pkl")
real.to_csv(OUTPUT_PATH + "data_real.csv", index=False)